In [1]:
from init import *

In [2]:
uuosapi.get_account(test_account1)

{'account_name': 'wkpmdjdsztyu',
 'head_block_num': 45474317,
 'head_block_time': '2021-01-12T02:25:42.000',
 'privileged': False,
 'last_code_update': '2021-01-12T02:24:33.000',
 'created': '2020-11-30T02:21:01.000',
 'core_liquid_balance': '75.6720 EOS',
 'ram_quota': 141621,
 'net_weight': 110000,
 'cpu_weight': 110000,
 'net_limit': {'used': 60800, 'available': 26297895, 'max': 26358695},
 'cpu_limit': {'used': 135920, 'available': 4891120, 'max': 5027040},
 'ram_usage': 95559,
 'permissions': [{'perm_name': 'active',
   'parent': 'owner',
   'required_auth': {'threshold': 1,
    'keys': [{'key': 'EOS4yV91cqR6db8ntKtCLFVwWfvvvDMZctpd85GsaX5GUMaiqUWYL',
      'weight': 1}],
    'accounts': [],
    'waits': []}},
  {'perm_name': 'owner',
   'parent': '',
   'required_auth': {'threshold': 1,
    'keys': [{'key': 'EOS7iPioB1Uscn14dYdw5PrqAHVU2F54Bi3kJ1qr6WbRe1tMtHGEo',
      'weight': 1}],
    'accounts': [],
    'waits': []}}],
 'total_resources': {'owner': 'wkpmdjdsztyu',
  'net_weig

In [4]:
a = {
    "account": test_account1,
    "permission": "active",
    "parent": "owner",
    "auth": {
        "threshold": 1,
        "keys": [
            {
                "key": "EOS4yV91cqR6db8ntKtCLFVwWfvvvDMZctpd85GsaX5GUMaiqUWYL",
                "weight": 1
            },
        ],
        "accounts": [{"permission":{"actor":test_account1,"permission":"eosio.code"},"weight":1}],
        "waits": []
    }
}
r = uuosapi.push_action('eosio', 'updateauth', a, {test_account1:'active'})
print('done!')

done!


In [6]:
test_account1

'wkpmdjdsztyu'

In [7]:
code = r'''
#include <vector>
#include <eosio/print.hpp>
#include <eosio/name.hpp>
#include <eosio/action.hpp>
#include <eosio/asset.hpp>

using namespace std;
using namespace eosio;

#define N(name) name##_n.value

struct transfer {
    name from;
    name to;
    asset amount;
    string memo;
    EOSLIB_SERIALIZE( transfer, (from)(to)(amount)(memo) )
};


extern "C" {
    void apply( uint64_t receiver, uint64_t code, uint64_t action ) {
        if (N("sayhello") == action) {
            eosio::print("hello,world\n");
            name sender("wkpmdjdsztyu");
            struct action a;
            a.account = "eosio.token"_n;
            a.name = "transfer"_n;
            a.authorization.push_back({sender, "active"_n});
            
            transfer t;
            t.from = sender;
            t.to = "eosio"_n;
            t.amount.amount = 10000;
            t.amount.symbol = symbol("EOS", 4);
            a.data = eosio::pack<transfer>(t);
            a.send();
        }
    }
}
'''
code = uuosapi.compile(test_account1, code, vm_type=0)
uuosapi.deploy_code(test_account1, code, vm_type=0)
print('done!')

done!


In [8]:
uuosapi.get_balance(test_account1), uuosapi.get_balance('eosio')

(75.672, 798712554.4884)

In [9]:
r = uuosapi.push_action(test_account1, 'sayhello', b'hello,world', {test_account1:'active'})
print(r['processed']['action_traces'][0]['console'])
print(r['processed']['elapsed'])

hello,world

1596


In [10]:
uuosapi.get_balance(test_account1), uuosapi.get_balance('eosio')

(74.672, 798712555.4884)